In [1]:
#Importing libraries
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from bs4 import BeautifulSoup
from urllib.request import urlopen
from datetime import datetime
import re
import pandas as pd
import time

In [9]:
#Getting the year page
worldcup_page0 = "https://firstskisport.com/cross-country/calendar.php?y=1920"

In [10]:
#Opening the webpage
worldcup_page0 = urlopen(worldcup_page0, timeout=10)

In [5]:
#BeautifulSoup the HTML for the page
worldcup_soup0 = BeautifulSoup(worldcup_page0, 'html.parser')
print(worldcup_soup0)

<!DOCTYPE html>

<html lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="Ski sport, Skisport, Cross-Country, Biathlon, Alpine, Ski Jumping, Nordic Combined, Winter manager, Ski fantasi, Fantaski" name="keywords">
<meta content="FirstSkisport.com. Visit ut for results, startlists, historical data and more." name="description"/>
<link href="img/icon.png" rel="apple-touch-icon"/>
<link href="img/icon.png" rel="icon" sizes="150x150"/>
<link href="img/icon.png" rel="shortcut icon" sizes="150x150"/>
<link href="https://firstskisport.com/img/favicon.png" rel="shortcut icon" type="image/png">
<link href="https://firstskisport.com/cross-country/calendar.php?y=1920" rel="canonical">
<meta content="aJP64wml7f9u4Djugoj3plq3O4YU3mxSw-SuKgGUSrw" name="google-site-verification"/>
<link href="../css/font.css" rel="stylesheet" type="text/css">
<link href="../lib/font-awesome-4.7.0

In [12]:
#Getting the links to each race in a season
worldcup_page1 = []
title_results_count = 0
for b in men_worldcup_soup0.find_all('a', {'title':'Results'}, href = True):
    if(title_results_count%2==0):
        worldcup_page1.append('https://firstskisport.com/cross-country/'+str(b['href']))
    title_results_count+=1
print(worldcup_page1)    


['https://firstskisport.com/cross-country/results.php?id=1808']


In [13]:
for a in range(len(men_worldcup_page1)):
    #BeautifulSouping the webpage
    worldcup_page = urlopen(men_worldcup_page1[a])

In [15]:
#Getting the race information and results
for a in range(len(worldcup_page1)):
    #BeautifulSouping the webpage
    worldcup_page = urlopen(worldcup_page1[a])
    worldcup_soup = BeautifulSoup(worldcup_page, 'html.parser')
    
    #Getting the race distance/technique and the city
    race_city = worldcup_soup.body.find('h1').text
    
    #Getting the date of the race and the country it takes place
    date_country = worldcup_soup.body.find('h2').text.split(", ")
    #Start by getting the date extracted
    date = str(date_country[2]).split(" ")[1:3]
    year = date[1]
    date = " ".join(date)
    #Now we need to get the date into yyyymmdd format
    date_obj = datetime.strptime(date, "%d.%b %Y")
    date = date_obj.strftime("%Y%m%d")
    
    #Now we get the country
    country = str(date_country[3])
    
    #Now the race and the city
    race_city = str(race_city).split("- ")
    race = race_city[0]
    city = race_city[1]
    
    #Now lets isolate the distance
    #Getting the mass start is optional
    distance = race.split(" ")[0]
    if(("Mass" in race) and ("Start" in distance)):
        ms=1
    else:
        ms=0
        
    #Start with relays.  Since for now we are only concerned with individual, we will not return
    if(distance.startswith("4x") or distance.startswith("3x")):
        distance = "Rel"
        
    #Now team sprints.  Also will not return
    if(distance=="Team"):
        distance = "Ts"
    
    #Now duathlon.  This won't be a thing until much later
    if(distance=="Duathlon"):
        technique = "P"
        #We want to make a list of all the important parts that will be
        #inputted into the Excel.  This table value will be returned later
        table = [date, city, country, distance, 1, technique]
    
    #Now sprints
    if(distance=="Sprint"):
        technique = race.split(" ")
        technique = technique[1][0]
        table = [date, city, country, distance, ms, technique]
    
    #And finally distance.  Here we will ask if the year is greater than 1985 and make sure it is not a relay
    #1985 is when freestyle came into play
    if(int(year) > 1985 and distance!="Rel"):
        technique = race.split(" ")
        technique = technique[2][0]
        table = [date, city, country, distance, ms, technique]
    else:
        table = [date, city, country, distance, ms, "N/A"]
    print(table)

['19200219', 'Holmenkollen', 'Norway', '50', 0, 'N/A']


In [19]:
#Getting the race information and results
for a in range(len(worldcup_page1)):
    #BeautifulSouping the webpage
    worldcup_page = urlopen(worldcup_page1[a])
    worldcup_soup = BeautifulSoup(worldcup_page, 'html.parser')
    
    #Getting the race distance/technique and the city
    race_city = worldcup_soup.body.find('h1').text
    print(race_city)
    
    #Getting the date of the race and the country it takes place
    date_country = worldcup_soup.body.find('h2').text.split(", ")
    #Start by getting the event and date extracted
    event = date_country[1]
    date = str(date_country[2]).split(" ")[1:3]
    year = date[1]
    date = " ".join(date)
    #Now we need to get the date into yyyymmdd format
    date_obj = datetime.strptime(date, "%d.%b %Y")
    date = date_obj.strftime("%Y%m%d")
    
    #Now we get the country
    country = str(date_country[3])
    
    #Now the race and the city
    race_city = str(race_city).split("- ")
    race = race_city[0]
    city = race_city[1]
    
    #Now lets isolate the distance
    #Getting the mass start is optional
    distance = race.split(" ")[0]
    if(("Mass" in race) and ("Start" in distance)):
        ms=1
    else:
        ms=0
        
    #Start with relays.  Since for now we are only concerned with individual, we will not return
    if(distance.startswith("4x") or distance.startswith("3x")):
        distance = "Rel"
        
    #Now team sprints.  Also will not return
    if(distance=="Team"):
        distance = "Ts"
    
    #Now duathlon.  This won't be a thing until much later
    if(distance=="Duathlon"):
        technique = "P"
        #We want to make a list of all the important parts that will be
        #inputted into the Excel.  This table value will be returned later
        table = [date, city, country, distance, 1, technique]
    
    #Now sprints
    if(distance=="Sprint"):
        technique = race.split(" ")
        technique = technique[1][0]
        table = [date, city, country, distance, ms, technique]
    
    #And finally distance.  Here we will ask if the year is greater than 1985 and make sure it is not a relay
    #1985 is when freestyle came into play
    if(int(year) > 1985 and distance!="Rel"):
        technique = race.split(" ")
        technique = technique[2][0]
        table = [date, city, country, distance, ms, technique]
    else:
        table = [date, city, country, distance, ms, "N/A"]

    #Now we get the skiers by looking at the the table
    body = worldcup_soup.body.find_all('table', {'class':'tablesorter sortTabell'})[0].find_all('td')
    #We want to get: places, skier, nation, ski_ids
    #Need to go through the body and extract this data
    #Skier's info comes up every seventh iteration
    places = []
    skier = []
    nation = []
    ski_ids = []
    for b in range(len(body)):
        if(b%7==0):
            #Making sure the skier didn't drop off, get dsq, etc.
            #This may be something to measure, but generally these events
            #would be bad to include in elo since they are so major and seemingly random
            #A randomness check could be done at a later time.
            if(str(body[b].text)!="DNF" and str(body[b].text)!="DSQ" and str(body[b].text)!="DNS"
                and str(body[b].text)!="DNQ" and str(body[b].text)!="OOT"):
                places.append(body[b].text) 
                ski_id = str(body[b+2])
                id_match = re.search(r'id=(\d+)', ski_id)
                ski_id = id_match.group(1)
                ski_ids.append(ski_id)
                skier_name = str(body[b+2])
                skier_match = re.search(r'title="([^"]+)"', skier_name)
                skier.append(skier_match.group(1))
                nation.append(body[b+4].text.strip())
    skiers = [places, skier, nation, ski_ids]
    #Using Pandas for creating a dataframe
    ski_df = pd.DataFrame(skiers).T
    ski_df.columns=['place', 'name', 'nation', 'id']                      

    #And now the table.  We are using a dictionary for this
    table_df = {'date': [table[0]]*len(ski_df),
               'city': [table[1]]*len(ski_df),
               'country': [table[2]]*len(ski_df),
               'distance': [table[3]]*len(ski_df),
               'ms': [table[4]]*len(ski_df),
               'technique': [table[5]]*len(ski_df)}
    
    #Concatinating table_df with ski_df
    ski_df = pd.concat([pd.DataFrame(table_df), ski_df], axis=1)
    
    #Correcting the column order
    column_order = ['date', 'city', 'country', 'distance', 'ms', 'technique',
                   'place', 'name', 'nation', 'id']
    ski_df = ski_df[column_order]
    print(ski_df)
                
    
        
    
    


50 km - Holmenkollen
Pre World Cup
       date          city country distance  ms technique place  \
0  19200219  Holmenkollen  Norway       50   0       N/A     1   
1  19200219  Holmenkollen  Norway       50   0       N/A     2   
2  19200219  Holmenkollen  Norway       50   0       N/A     3   

                   name  nation     id  
0         Thorleif Haug  Norway  10000  
1  Johan Grøttumsbråten  Norway    879  
2        Torkel Persson  Sweden    881  


In [2]:
#Function to get results from the worldcup_page
def get_results(worldcup_page):
        #BeautifulSouping the webpage
    worldcup_page = urlopen(worldcup_page)
    worldcup_soup = BeautifulSoup(worldcup_page, 'html.parser')

    #Getting the race distance/technique and the city
    race_city = worldcup_soup.body.find('h1').text

    #Getting the date of the race and the country it takes place
    date_country = worldcup_soup.body.find('h2').text.split(", ")
    event = date_country[1]
    #Start by getting the event and date extracted.  The try and except are because of a weird exception in 1980.
    date = str(date_country[2]).split(" ")[1:3]
    try:
        year = date[1]
        date = " ".join(date)
        #Now we need to get the date into yyyymmdd format
        date_obj = datetime.strptime(date, "%d.%b %Y")
        date = date_obj.strftime("%Y%m%d")
    
    #There is a women's race in 1980 that does not have a date 
    except:
        year = "1980"
        date = "19800101"

    print(date)
    #Now we get the country
    country = str(date_country[3])

    #Now the race and the city
    race_city = str(race_city).split("- ")
    race = race_city[0]
    city = race_city[1]

    #Now lets isolate the distance
    #Getting the mass start is optional
    distance = race.split(" ")[0]
    if(("Mass" in race) and ("Start" in distance)):
        ms=1
    else:
        ms=0

    #Start with relays.  Since for now we are only concerned with individual, we will not return
    if(distance.startswith("4x") or distance.startswith("3x")):
        distance = "Rel"

    #Now team sprints.  Also will not return
    if(distance=="Team"):
        distance = "Ts"

    #Now duathlon.  This won't be a thing until much later
    if(distance=="Duathlon"):
        technique = "P"
        #We want to make a list of all the important parts that will be
        #inputted into the Excel.  This table value will be returned later
        table = [date, city, country, event, distance, 1, technique]

    #Now sprints
    elif(distance=="Sprint"):
        technique = race.split(" ")
        technique = technique[1][0]
        table = [date, city, country, event, distance, ms, technique]
    
    elif(distance=="Stage"):
        technique = "N/A"
        table = [date, city, country, event, distance, ms, technique]
    #And finally distance.  Here we will ask if the year is greater than 1985 and make sure it is not a relay
    #1985 is when freestyle came into play
    elif(int(year) > 1985 and distance!="Rel"):
        technique = race.split() #Empty splits all blank space.
        technique = technique[2][0]
        table = [date, city, country, event, distance, ms, technique]
    else:
        table = [date, city, country, event, distance, ms, "N/A"]

    #Now we get the skiers by looking at the the table
    body = worldcup_soup.body.find_all('table', {'class':'tablesorter sortTabell'})[0].find_all('td')
    #We want to get: places, skier, nation, ski_ids
    #Need to go through the body and extract this data
    #Skier's info comes up every seventh iteration
    places = []
    skier = []
    nation = []
    ski_ids = []
    for b in range(len(body)):
        if(b%7==0):
            #Making sure the skier didn't drop off, get dsq, etc.
            #This may be something to measure, but generally these events
            #would be bad to include in elo since they are so major and seemingly random
            #A randomness check could be done at a later time.
            if(str(body[b].text)!="DNF" and str(body[b].text)!="DSQ" and str(body[b].text)!="DNS"
                and str(body[b].text)!="DNQ" and str(body[b].text)!="OOT"):
                places.append(body[b].text) 
                ski_id = str(body[b+2])
                id_match = re.search(r'id=(\d+)', ski_id)
                ski_id = id_match.group(1)
                ski_ids.append(ski_id)
                skier_name = str(body[b+2])
                skier_match = re.search(r'title="([^"]+)"', skier_name)
                skier.append(skier_match.group(1))
                nation.append(body[b+4].text.strip())
    skiers = [places, skier, nation, ski_ids]
    #Using Pandas for creating a dataframe
    ski_df = pd.DataFrame(skiers).T
    ski_df.columns=['Place', 'Name', 'Nation', 'ID']                      

    #And now the table.  We are using a dictionary for this
    table_df = {'Date': [table[0]]*len(ski_df),
               'City': [table[1]]*len(ski_df),
               'Country': [table[2]]*len(ski_df),
               'Event': [table[3]]*len(ski_df),
               'Distance': [table[4]]*len(ski_df),
               'MS': [table[5]]*len(ski_df),
               'Technique': [table[6]]*len(ski_df)}

    #Concatinating table_df with ski_df
    ski_df = pd.concat([pd.DataFrame(table_df), ski_df], axis=1)

    #Correcting the column order
    column_order = ['Date', 'City', 'Country','Event', 'Distance', 'MS', 'Technique',
                   'Place', 'Name', 'Nation', 'ID']
    ski_df = ski_df[column_order]
    return ski_df     
    







In [3]:
start_time = time.time()
#0 is men, 1 is women
for a in range(0,2):
    if(a==0):
        sheet = "Men"
    else:
        sheet = "Ladies"
#Start with an empty dataframe with the columns we want
    ski_df = pd.DataFrame(columns = ['Date', 'City', 'Country','Event', 'Distance', 'MS', 'Technique',
                   'Place', 'Name', 'Nation', 'ID'])
    for b in range(1920, 2025):
        print(b)
        #Getting the year page
        if(a==0):
            worldcup_page0 = "https://firstskisport.com/cross-country/calendar.php?y=" +str(b)
        else:
            worldcup_page0 = "https://firstskisport.com/cross-country/calendar.php?y=" +str(b) + "&g=w"
        #Opening the webpage
        worldcup_page0 = urlopen(worldcup_page0, timeout=10)   
        #BeautifulSoup the HTML for the page
        worldcup_soup0 = BeautifulSoup(worldcup_page0, 'html.parser')

        #Getting the links to each race in a season
        worldcup_page1 = []
        title_results_count = 0
        for c in worldcup_soup0.find_all('a', {'title':'Results'}, href = True):
            if(title_results_count%2==0):
                worldcup_page1.append('https://firstskisport.com/cross-country/'+str(c['href']))
            title_results_count+=1

        #Going through each race in a season and getting the result.
        for c in range(len(worldcup_page1)):
            result = get_results(worldcup_page1[c])
            result['Race'] = c+1
            result['Season'] = b
            ski_df = pd.concat([ski_df, result], axis=0)
           
        
        
        
            
            
    #Saving it to a specific excel worksheet
    if(a==0):
        ski_df['Sex'] = "M"
        men_df = ski_df
        with pd.ExcelWriter("/Users/syverjohansen/ski/elo/python/ski/excel365/all_demo.xlsx", 
                            engine='xlsxwriter') as writer:
            ski_df.to_excel(writer, sheet_name=sheet, index=False)
    else:
        ski_df['Sex'] = "L"
        ladies_df = ski_df
        with pd.ExcelWriter("/Users/syverjohansen/ski/elo/python/ski/excel365/all_demo.xlsx", 
                            engine='openpyxl', mode='a') as writer:
            ski_df.to_excel(writer, sheet_name=sheet, index=False)
#Combine men and ladies dfs and write to a pickle


print(time.time() - start_time)
    
    
    

1920
19200219
1921
1922
19220223
1923
19230222
1924
19240130
19240202
1925
19250212
19250214
1926
19260204
19260207
1927
19270203
19270205
1928
19280214
19280217
1929
19290207
19290209
1930
19300227
19300303
1931
19310213
19310215
1932
19320210
19320213
1933
19330210
19330212
19330212
1934
19340222
19340224
19340225
1935
19350215
19350217
19350218
1936
19360210
19360212
19360215
1937
19370214
19370216
19370218
19370224
1938
19380224
19380226
19380228
19380307
1939
19390215
19390217
19390219
19390306
1940
19400228
1941
1942
1943
1944
1945
1946
19460227
1947
19470226
1948
19480131
19480203
19480206
19480303
1949
19490302
1950
19500203
19500205
19500206
19500301
1951
19510224
1952
19520218
19520220
19520223
19520308
1953
19530228
1954
19540216
19540218
19540220
19540221
19540306
1955
19550305
1956
19560127
19560130
19560202
19560204
19560225
1957
19570302
1958
19580302
19580304
19580306
19580308
19580315
1959
19590307
1960
19600219
19600223
19600225
19600227
19600319
1961
19610311
1962
19